In [ ]:
# default_exp database

In [ ]:
#hide
from nbdev.showdoc import *

# database

> This module is used to collect, maintain, and export all relevant information (= the _database_).

In [ ]:
#export
import pandas as pd
from pathlib import Path
from typing import List, Tuple, Dict, Optional

In [ ]:
class Database:
    
    def __init__(self) -> None:
        self.configs = Configs()


In [ ]:
class Configs:
    
    def __init__(self) -> None:
        self.set_default_configs()
        
    def set_default_configs(self) 
        
    